# Лабораторная работа №1
## Анализ социальных сетей

**Цель:** Овладеть `igraph` для моделирования, визуализации и анализа.

---

## 1. Теоретические положения

Социальная сеть — граф $G = (V, E)$.

### 1.1. Структурные метрики
- **Плотность**: $\Delta = \frac{2L}{g(g-1)}$ (ненапр.), $\Delta = \frac{L}{g(g-1)}$ (напр.)
- **Кластеризация**: $C = \frac{3\tau}{\pi}$
- **Диаметр**: $D = \max d(i,j)$

### 1.2. Центральность
- $C'_D(i) = \frac{\deg(i)}{g-1}$
- $C'_C(i) = \frac{g-1}{\sum d(i,j)}$
- $C'_B(i) = \frac{2\sum \frac{\sigma_{jk}(i)}{\sigma_{jk}}}{(g-1)(g-2)}$

### 1.3. Престиж
- $P'_D(i) = \frac{\deg_{\text{in}}(i)}{g-1}$
- $P'_P(i) = \frac{|I_i|}{g-1} \cdot \frac{g-1}{\sum_{j\in I_i} d(j,i)}$
- $P_R$: решение $(\frac{1}{a}I - X^T)\mathbf{p} = \mathbf{x}$

### 1.4. Укладка
- **FR**: $E = \sum \frac{1}{r^2} + \sum r^3$ → кластеры
- **KK**: $E = \sum k_{ij}(\delta_{ij} - l_{ij})^2$ → топология

---

## 2. Подготовка

In [2]:
if (!require("igraph")) install.packages("igraph")
library(igraph)

setwd("C:/rlab/lab1")
cat("Рабочая папка:", getwd(), "\n")

ERROR: Error in setwd("C:/rlab/lab1"): cannot change working directory


---
# ЧАСТЬ A: Ненаправленная сеть — `florentine_business.csv`

### Пункт 1: Извлечение социоматрицы

In [3]:
A_flor <- as.matrix(read.csv("florentine_business.csv", header = FALSE, skip = 2))
mode(A_flor) <- "numeric"
cat("Пункт 1: Социоматрица (16×16)\n")
print(A_flor)

Warning message in file(file, "rt"):
“cannot open file 'florentine_business.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


### Пункт 2: Преобразование в список рёбер

In [ ]:
g_flor <- graph_from_adjacency_matrix(A_flor, mode = "max", diag = FALSE)
edge_list <- as_edgelist(g_flor)
cat("Пункт 2: Первые 5 рёбер\n")
print(head(edge_list, 5))

### Пункт 3: Присвоение имён

In [ ]:
fam_names <- read.csv("florentine_business.csv", header = FALSE, nrows = 1, skip = 1)[1,]
fam_names <- as.character(unlist(fam_names))
V(g_flor)$name <- fam_names
rownames(A_flor) <- colnames(A_flor) <- fam_names
cat("Пункт 3: Социоматрица с именами\n")
print(A_flor)

### Пункт 4: Построение ненаправленной сети

In [ ]:
g_flor <- graph_from_adjacency_matrix(A_flor, mode = "undirected", weighted = NULL, diag = FALSE)
cat("Пункт 4: Граф построен (undirected, бинарный, без петель)\n")

### Пункт 5: Визуализация

In [ ]:
par(mfrow = c(1, 2))
plot(g_flor, layout = layout_with_fr, main = "FR", vertex.label = V(g_flor)$name, vertex.size = 15)
plot(g_flor, layout = layout_with_kk, main = "KK", vertex.label = V(g_flor)$name, vertex.size = 15)
par(mfrow = c(1, 1))
cat("Пункт 5: Визуализация завершена\n")

### Пункт 6: Описательная сводка

In [ ]:
cat("Пункт 6: Сводка\n")
summary(g_flor)

### Пункт 7: Пятичисловая сводка

In [ ]:
size <- vcount(g_flor)
edges <- ecount(g_flor)
max_edges <- size * (size - 1) / 2
density <- edges / max_edges
components <- igraph::components(g_flor)$no
diameter_val <- diameter(g_flor)
clustering <- transitivity(g_flor, type = "global")

cat("Пункт 7: Пятичисловая сводка\n")
cat("• Размер: ", size, "\n")
cat("• Плотность: ", round(density, 4), "\n")
cat("• Компонент: ", components, "\n")
cat("• Диаметр: ", diameter_val, "\n")
cat("• Кластеризация: ", round(clustering, 4), "\n")

### Пункт 8: Меры центральности

In [ ]:
non_isol <- V(g_flor)[degree(g_flor) > 0]
deg <- degree(g_flor, v = non_isol, normalized = TRUE)
close <- closeness(g_flor, v = non_isol, normalized = TRUE)
betw <- betweenness(g_flor, v = non_isol, normalized = TRUE)
eig <- eigen_centrality(g_flor)$vector[non_isol]

cent_table <- data.frame(
  Семья = non_isol$name,
  C_D = round(deg, 4),
  C_C = round(close, 4),
  C_B = round(betw, 4),
  C_I = round(eig, 4)
)

cat("Пункт 8: Центральность\n")
print(cent_table, row.names = FALSE)

---
# ЧАСТЬ B: Направленная сеть — `krackhardt_advice.csv`

### Пункт 1: Извлечение социоматрицы

In [ ]:
A_adv <- as.matrix(read.csv("krackhardt_advice.csv", header = FALSE, skip = 1))
mode(A_adv) <- "numeric"
cat("Пункт 1 (напр.): Социоматрица (21×21)\n")
print(A_adv)

### Пункт 2: Преобразование в список рёбер

In [ ]:
g_adv <- graph_from_adjacency_matrix(A_adv, mode = "directed")
edge_list_dir <- as_edgelist(g_adv)
cat("Пункт 2 (напр.): Первые 5 рёбер\n")
print(head(edge_list_dir, 5))

### Пункт 3: Присвоение имён

In [ ]:
mgr_names <- paste("Manager", 1:21)
V(g_adv)$name <- mgr_names
rownames(A_adv) <- colnames(A_adv) <- mgr_names
cat("Пункт 3 (напр.): Имена присвоены\n")

### Пункт 4: Построение направленной сети

In [ ]:
g_adv <- graph_from_adjacency_matrix(A_adv, mode = "directed", diag = FALSE)
cat("Пункт 4 (напр.): Граф построен (directed)\n")

### Пункт 5: Визуализация

In [ ]:
par(mfrow = c(1, 2))
plot(g_adv, layout = layout_with_fr, main = "FR (советы)", vertex.label = V(g_adv)$name, edge.arrow.size = 0.4)
plot(g_adv, layout = layout_with_kk, main = "KK (советы)", vertex.label = V(g_adv)$name, edge.arrow.size = 0.4)
par(mfrow = c(1, 1))
cat("Пункт 5 (напр.): Визуализация завершена\n")

### Пункт 6: Описательная сводка

In [ ]:
cat("Пункт 6 (напр.): Сводка\n")
summary(g_adv)

### Пункт 7: Пятичисловая сводка

In [ ]:
size_dir <- vcount(g_adv)
edges_dir <- ecount(g_adv)
max_edges_dir <- size_dir * (size_dir - 1)
density_dir <- edges_dir / max_edges_dir
weak_comp <- igraph::components(g_adv, mode = "weak")$no
strong_comp <- igraph::components(g_adv, mode = "strong")$no
diameter_dir <- diameter(g_adv, directed = TRUE)
clustering_dir <- transitivity(g_adv, type = "global")

cat("Пункт 7 (напр.): Пятичисловая сводка\n")
cat("• Размер: ", size_dir, "\n")
cat("• Плотность: ", round(density_dir, 4), "\n")
cat("• Слабых компонент: ", weak_comp, "\n")
cat("• Сильных компонент: ", strong_comp, "\n")
cat("• Диаметр: ", diameter_dir, "\n")
cat("• Кластеризация: ", round(clustering_dir, 4), "\n")

### Пункт 10: Меры престижа

In [ ]:
P_D <- degree(g_adv, mode = "in", normalized = TRUE)

# Proximity
reach <- colSums(distances(g_adv, mode = "in") < Inf) - 1
P_P <- (reach / (size_dir - 1)) * closeness(g_adv, mode = "in", normalized = TRUE)

# Rank
g_rev <- reverse_edges(g_adv)
P_R <- eigen_centrality(g_rev)$vector
P_R <- P_R / sum(P_R)

prestige_table <- data.frame(
  Менеджер = mgr_names,
  P_D = round(P_D, 4),
  P_P = round(P_P, 4),
  P_R = round(P_R, 4)
)

cat("Пункт 10: Престиж\n")
print(prestige_table, row.names = FALSE)

---
# Пункт 11: Общий вывод

In [ ]:
cat(strrep("=", 60), "\n")
cat("ПУНКТ 11: ОБЩИЙ ВЫВОД\n")
cat(strrep("=", 60), "\n")
cat("• Флорентийские семьи: фрагментирована (6 компонент), низкая плотность (0.125), но Medici и Barbadori — ключевые.\n")
cat("• Менеджеры: связная, плотная (0.452), кластеризованная (0.664), диаметр 3 — идеальна для распространения.\n")
cat("• FR: выделяет кластеры; KK: сохраняет расстояния.\n")
cat("• Центральность и престиж — мощные инструменты выявления лидеров.\n")
cat(strrep("=", 60), "\n")